## Extract Parallel Texts from TED talks

Derived / inspired by : [Ajinkya Kulkarni's GitHub](https://github.com/ajinkyakulkarni14/How-I-Extracted-TED-talks-for-parallel-Corpus-/blob/master/Ipython_notebook.ipynb).

In [ ]:
#import pandas as pd
from bs4 import BeautifulSoup
#import urllib
import requests
#import shutil
#import codecs
import os, glob
import csv

In [ ]:
def enlist_talk_names(url, dict_):
    #r = urllib.urlopen(url).read()
    #soup = BeautifulSoup(r)
    r = requests.get(url)
    print("  Got %d bytes from %s" % (len(r.text), url))
    soup = BeautifulSoup(r.text, 'html.parser')
    talks= soup.find_all("a",class_='')
    for i in talks:
        if i.attrs['href'].find('/talks/')==0 and dict_.get(i.attrs['href'])!=1:
            dict_[i.attrs['href']]=1
    return dict_

In [ ]:
all_talk_names={}

# Get all pages of talks (seems a bit abusive)
#for i in xrange(1,61):
#    url='https://www.ted.com/talks?page=%d'%(i)
#    all_talk_names=enlist_talk_names(url, all_talk_names)

# Just the AI topic talks
url='https://www.ted.com/talks?sort=newest&q=ai'
all_talk_names=enlist_talk_names(url,all_talk_names)
len(all_talk_names)

In [ ]:
len(all_talk_names)

In [ ]:
data_path = './data'
if not os.path.exists(data_path):
    os.makedirs(data_path)

def extract_talk(url, talk_name, language_list=['en', 'ko', 'ja']):
    r = requests.get(url)
    print("  Got %d bytes from %s" % (len(r.text), url))
    soup = BeautifulSoup(r.text, 'html.parser')
    #df=pd.DataFrame()
    talk_data = dict() # This is indexed on [timestep][language]
    for i in soup.findAll('link'):
        if i.get('href')!=None and i.attrs['href'].find('?language=')!=-1:
            #print i.attrs['href']
            lang=i.attrs['hreflang']
            url_lang=i.attrs['href']
            if not lang in language_list:
                continue
            r_lang = requests.get(url_lang)
            print("    Lang[%s] : Got %d bytes" % (lang, len(r_lang.text), ))
            lang_soup = BeautifulSoup(r_lang.text, 'html.parser')

            for i in lang_soup.findAll('span',class_='talk-transcript__fragment'):
                ts = i.attrs['data-time']
                if ts not in talk_data: 
                    talk_data[ts]=dict()
                talk_data[ts][lang] = i.text.replace('\n',' ')

    # Now flatten out the talk_data into time_step order
    talk_data_csv = [ ['ts']+language_list, ]
    for ts in sorted(talk_data.keys(), key=int):
        row = [ts] + [ talk_data[ts].get(lang, '') for lang in language_list]
        talk_data_csv.append(row)
        
    with open(os.path.join(data_path, talk_name+'.csv'), 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(talk_data_csv)

In [ ]:
for name in all_talk_names:
    extract_talk('https://www.ted.com'+name+'/transcript', name[7:])
    break